In [3]:
import torch
import time
from torch import autograd
#GPU加速
print(torch.__version__)
print(torch.cuda.is_available())

a=torch.randn(10000,1000)
b=torch.randn(1000,10000)
print(a)
print(b)
t0=time.time()
c=torch.matmul(a,b)
t1=time.time()

print(a.device,t1-t0,c.norm(2))

device=torch.device('cuda')
print(device)
a=a.to(device)
b=b.to(device)

t0=time.time()
c=torch.matmul(a,b)
t2=time.time()
print(a.device,t2-t0,c.norm(2))


t0=time.time()
c=torch.matmul(a,b)
t2=time.time()

print(a.device,t2-t0,c.norm(2))

1.13.0
True
tensor([[ 1.2955, -1.5694,  0.5075,  ..., -0.2906, -0.1682, -0.5837],
        [ 2.5299,  1.0550, -0.8601,  ...,  0.6747, -2.1697, -0.9547],
        [ 1.4518, -0.7689,  0.5799,  ..., -0.3670,  1.9537, -2.3454],
        ...,
        [-1.2146,  0.7101,  0.4965,  ...,  0.5240, -0.3242,  0.7821],
        [ 0.9410, -1.0559, -1.0886,  ..., -0.2211, -0.8319,  3.8158],
        [-0.2719,  1.0463,  0.6932,  ...,  1.4020, -0.3865,  0.9531]])
tensor([[-7.6368e-01,  5.4533e-04,  6.3869e-01,  ...,  1.9057e-01,
          3.5469e-01,  2.1302e-01],
        [ 5.0872e-02, -1.9517e-01,  2.0536e-01,  ..., -2.6769e-01,
          9.0115e-01, -1.6780e+00],
        [ 1.2869e+00, -1.9171e-01, -1.4041e+00,  ...,  7.1393e-01,
          1.7104e+00, -7.8553e-01],
        ...,
        [ 1.1021e+00,  6.7650e-01, -1.6953e-01,  ...,  5.6324e-01,
         -3.3188e-01,  2.0026e+00],
        [ 3.1847e-01, -2.7512e-01,  1.6981e+00,  ..., -1.8286e-01,
         -1.1563e-01,  8.1707e-01],
        [ 1.5270e+00, -1.3